# Mapping Signal Groups

In [33]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import xarray as xr
import zarr
import json
import pandas as pd
from pathlib import Path
from src.archive.transforms import PipelineRegistry

def load_source(path):
    store = zarr.open(path)
    keys = list(store.keys())
    dt = {key: xr.open_zarr(path + '/'+ key) for key in keys}
    return dt
    

def make_channel_df(dataset):
    items = []
    for key, ds in dataset.data_vars.items():
        item = dict(name=ds.attrs['uda_name'], description=ds.attrs['description'])
        items.append(item)

    df = pd.DataFrame(items)
    return df

def group_signals_by_source(source, dt) -> xr.Dataset:
    datasets = []
    for name, ds in dt.items():
        ds = ds.squeeze()

        # Drop error if all zeros
        if (ds['error'].values == 0).all():
            ds = ds.drop_vars('error')

        # Rename variables
        new_names = {}
        if 'error' in ds:
            new_names['data'] = '_'.join([name, 'data'])
            new_names['error'] = '_'.join([name, 'error'])
        else:
            name = name + '_' if name == 'time' else name
            new_names['data'] = name

        ds = ds.rename(new_names)

        # Update attributes
        attrs = ds.attrs
        attrs['name'] = source + '/' + new_names['data']
        ds[new_names['data']].attrs = attrs
        datasets.append(ds)

    dataset = xr.merge(datasets)
    dataset.attrs = {}

    return dataset

def tensorise_group(dataset: xr.Dataset, stem: str) -> xr.Dataset:
    dim_name = f'{stem}_channel'
    group_keys = [key for key in dataset.data_vars.keys() if stem in key]
    channels = [dataset[key] for key in group_keys]
    dataset[stem] = xr.combine_nested(channels, concat_dim=dim_name)
    dataset[stem] = dataset[stem].assign_coords({dim_name: group_keys})
    dataset[stem] = dataset[stem].chunk('auto')
    dataset = dataset.drop_vars(group_keys)
    return dataset

pipelines = PipelineRegistry()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### ABM - Needs Review

In [35]:
source = 'abm'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 2MB
Dimensions:             (dim_0: 32, time: 7500, chord: 11)
Coordinates:
  * dim_0               (dim_0) float32 128B 1.0 2.0 3.0 4.0 ... 30.0 31.0 32.0
  * time                (time) float32 30kB -0.1 -0.0996 -0.0992 ... 2.899 2.9
  * chord               (chord) float32 44B 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0
Data variables: (12/21)
    abm/calib_shot      int16 2B dask.array<chunksize=(), meta=np.ndarray>
    abm/channel_status  (dim_0) float32 128B dask.array<chunksize=(32,), meta=np.ndarray>
    abm/channel_type    (dim_0) float32 128B dask.array<chunksize=(32,), meta=np.ndarray>
    abm/gain            (dim_0) float32 128B dask.array<chunksize=(32,), meta=np.ndarray>
    abm/i-bol           (time, dim_0) float32 960kB dask.array<chunksize=(3750, 16), meta=np.ndarray>
    abm/km              (dim_0) float32 128B dask.array<chunksize=(32,), meta=np.ndarray>
    ...                  ...
    abm/taum            (dim_0) float32 128B dask.array<chunksize=(32,), meta=np.ndarray>
    abm/tco_emis        (time, chord) float32 330kB dask.array<chunksize=(3750, 11), meta=np.ndarray>
    abm/v-bol           (time, dim_0) float32 960kB dask.array<chunksize=(3750, 16), meta=np.ndarray>
    abm/version         float32 4B dask.array<chunksize=(), meta=np.ndarray>
    abm/z-chords        (dim_0) float32 128B dask.array<chunksize=(32,), meta=np.ndarray>
    abm/z-slits         (dim_0) float32 128B dask.array<chunksize=(32,), meta=np.ndarray>

### ADA

In [54]:
source = 'ada'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
# dataset = group_signals_by_source(source, dt)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 23MB
Dimensions:                 (time: 887, dim_0: 2179)
Coordinates:
  * time                    (time) float32 4kB -0.09717 -0.09512 ... 0.6962
  * dim_0                   (dim_0) float32 9kB -0.2456 -0.2447 ... 1.608 1.609
Data variables:
    ada/dalpha_integrated   (time) float32 4kB dask.array<chunksize=(887,), meta=np.ndarray>
    ada/dalpha_inverted     (time, dim_0) float32 8MB dask.array<chunksize=(683, 1405), meta=np.ndarray>
    ada/dalpha_peak_radius  (time) float32 4kB dask.array<chunksize=(887,), meta=np.ndarray>
    ada/dalpha_raw_full     (time, dim_0) float32 8MB dask.array<chunksize=(683, 719), meta=np.ndarray>
    ada/geo_full            (time, dim_0) float32 8MB dask.array<chunksize=(887, 2179), meta=np.ndarray>
    ada/inner_integrated    (time) float32 4kB dask.array<chunksize=(887,), meta=np.ndarray>
    ada/inner_peak_radius   (time) float32 4kB dask.array<chunksize=(887,), meta=np.ndarray>
    ada/passnumber          (time) float32 4kB dask.array<chunksize=(887,), meta=np.ndarray>
    ada/status              (time) float32 4kB dask.array<chunksize=(887,), meta=np.ndarray>

### ADG

In [55]:
source = 'adg'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 8kB
Dimensions:                (time: 389)
Coordinates:
  * time                   (time) float32 2kB -0.09512 -0.09307 ... 0.6982
Data variables:
    adg/density_gradient   (time) float32 2kB dask.array<chunksize=(389,), meta=np.ndarray>
    adg/gradient_position  (time) float32 2kB dask.array<chunksize=(389,), meta=np.ndarray>
    adg/passnumber         (time) float32 2kB dask.array<chunksize=(389,), meta=np.ndarray>
    adg/status             (time) float32 2kB dask.array<chunksize=(389,), meta=np.ndarray>

### AGA

In [56]:
source = 'aga'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 15MB
Dimensions:             (time: 132812)
Coordinates:
  * time                (time) float32 531kB -2.5 -2.5 -2.5 ... 10.2 10.2 10.2
Data variables: (12/25)
    aga/bc11            (time) float32 531kB dask.array<chunksize=(132812,), meta=np.ndarray>
    aga/bc5             (time) float32 531kB dask.array<chunksize=(132812,), meta=np.ndarray>
    aga/fig             (time) float32 531kB dask.array<chunksize=(69312,), meta=np.ndarray>
    aga/fig_raw         (time) float32 531kB dask.array<chunksize=(69312,), meta=np.ndarray>
    aga/hecc            (time) float32 531kB dask.array<chunksize=(132812,), meta=np.ndarray>
    aga/helios          (time) float32 531kB dask.array<chunksize=(132812,), meta=np.ndarray>
    ...                  ...
    aga/outboard_total  (time) float32 531kB dask.array<chunksize=(132812,), meta=np.ndarray>
    aga/passnumber      (time) float64 1MB dask.array<chunksize=(132812,), meta=np.ndarray>
    aga/status          (time) float64 1MB dask.array<chunksize=(132812,), meta=np.ndarray>
    aga/tc11            (time) float32 531kB dask.array<chunksize=(132812,), meta=np.ndarray>
    aga/tc5a            (time) float32 531kB dask.array<chunksize=(132812,), meta=np.ndarray>
    aga/tc5b            (time) float32 531kB dask.array<chunksize=(132812,), meta=np.ndarray>

### AHX

In [57]:
source = 'ahx'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 420kB
Dimensions:         (time: 15001)
Coordinates:
  * time            (time) float32 60kB -0.1 -0.0999 -0.0998 ... 1.4 1.4 1.4
Data variables:
    ahx/hxr_mezzw   (time) float32 60kB dask.array<chunksize=(15001,), meta=np.ndarray>
    ahx/hxr_s       (time) float32 60kB dask.array<chunksize=(15001,), meta=np.ndarray>
    ahx/hxr_w       (time) float32 60kB dask.array<chunksize=(15001,), meta=np.ndarray>
    ahx/passnumber  (time) float32 60kB dask.array<chunksize=(15001,), meta=np.ndarray>
    ahx/status      (time) float32 60kB dask.array<chunksize=(15001,), meta=np.ndarray>
    ahx/version     (time) float32 60kB dask.array<chunksize=(15001,), meta=np.ndarray>

### AIM

In [59]:
source = 'aim'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 2MB
Dimensions:              (time: 50000)
Coordinates:
  * time                 (time) float32 200kB -0.01 -0.00998 ... 0.99 0.99
Data variables:
    aim/da_hm10_t_data   (time) float32 200kB dask.array<chunksize=(50000,), meta=np.ndarray>
    aim/da_hm10_t_error  (time) float32 200kB dask.array<chunksize=(50000,), meta=np.ndarray>
    aim/da_to10_data     (time) float32 200kB dask.array<chunksize=(50000,), meta=np.ndarray>
    aim/da_to10_error    (time) float32 200kB dask.array<chunksize=(50000,), meta=np.ndarray>
    aim/passnumber       (time) float64 400kB dask.array<chunksize=(50000,), meta=np.ndarray>
    aim/status           (time) float64 400kB dask.array<chunksize=(50000,), meta=np.ndarray>

### AIR

In [60]:
source = 'air'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 15MB
Dimensions:                   (time: 3121, dim_0: 299)
Coordinates:
  * time                      (time) float32 12kB -0.04998 -0.04978 ... 0.4088
  * dim_0                     (dim_0) float32 1kB 0.8 0.8016 ... 1.269 1.271
Data variables: (12/31)
    air/alphaconst_osp        (time) float32 12kB dask.array<chunksize=(3121,), meta=np.ndarray>
    air/alphaconst_osp_elm    (time) float32 12kB dask.array<chunksize=(3121,), meta=np.ndarray>
    air/camera_view_osp       (time) float32 12kB dask.array<chunksize=(3121,), meta=np.ndarray>
    air/etot_osp              (time) float32 12kB dask.array<chunksize=(3121,), meta=np.ndarray>
    air/etot_osp_elm          (time) float32 12kB dask.array<chunksize=(3121,), meta=np.ndarray>
    air/etotsum_osp           (time) float32 12kB dask.array<chunksize=(3121,), meta=np.ndarray>
    ...                        ...
    air/status                (time) float32 12kB dask.array<chunksize=(3121,), meta=np.ndarray>
    air/svn_revision          (time) float32 12kB dask.array<chunksize=(3121,), meta=np.ndarray>
    air/temperature_osp       (time) float32 12kB dask.array<chunksize=(3121,), meta=np.ndarray>
    air/tprofile_osp          (time, dim_0) float32 4MB dask.array<chunksize=(860, 150), meta=np.ndarray>
    air/z_extent_osp          (time) float32 12kB dask.array<chunksize=(3121,), meta=np.ndarray>
    air/z_start_osp           (time) float32 12kB dask.array<chunksize=(3121,), meta=np.ndarray>

### AIT

In [61]:
source = 'ait'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 22MB
Dimensions:                   (time: 3162, dim_0: 214)
Coordinates:
  * time                      (time) float32 13kB -0.04995 -0.04975 ... 0.4089
  * dim_0                     (dim_0) float32 856B 0.8 0.8063 ... 1.815 1.82
Data variables: (12/59)
    ait/alphaconst_isp        (time) float32 13kB dask.array<chunksize=(3162,), meta=np.ndarray>
    ait/alphaconst_isp_elm    (time) float32 13kB dask.array<chunksize=(3162,), meta=np.ndarray>
    ait/alphaconst_osp        (time) float32 13kB dask.array<chunksize=(3162,), meta=np.ndarray>
    ait/alphaconst_osp_elm    (time) float32 13kB dask.array<chunksize=(3162,), meta=np.ndarray>
    ait/camera_view_isp       (time) float32 13kB dask.array<chunksize=(3162,), meta=np.ndarray>
    ait/camera_view_osp       (time) float32 13kB dask.array<chunksize=(3162,), meta=np.ndarray>
    ...                        ...
    ait/tprofile_isp          (time, dim_0) float32 3MB dask.array<chunksize=(1648, 158), meta=np.ndarray>
    ait/tprofile_osp          (time, dim_0) float32 3MB dask.array<chunksize=(1648, 51), meta=np.ndarray>
    ait/z_extent_isp          (time) float32 13kB dask.array<chunksize=(3162,), meta=np.ndarray>
    ait/z_extent_osp          (time) float32 13kB dask.array<chunksize=(3162,), meta=np.ndarray>
    ait/z_start_isp           (time) float32 13kB dask.array<chunksize=(3162,), meta=np.ndarray>
    ait/z_start_osp           (time) float32 13kB dask.array<chunksize=(3162,), meta=np.ndarray>

### ALP - needs review

In [50]:
source = 'alp'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

pipelines = PipelineRegistry()
dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 3MB
Dimensions:                        (time: 288, npts_inner: 32, npts_outer: 64)
Coordinates:
  * time                           (time) float32 1kB 0.1504 0.1514 ... 0.4489
Dimensions without coordinates: npts_inner, npts_outer
Data variables: (12/105)
    alp/badamps                    float32 4B dask.array<chunksize=(), meta=np.ndarray>
    alp/inner_lo_chisq             (time, npts_inner) float32 37kB dask.array<chunksize=(288, 32), meta=np.ndarray>
    alp/inner_lo_denpeakval        (time) float32 1kB dask.array<chunksize=(288,), meta=np.ndarray>
    alp/inner_lo_density           (time, npts_inner) float32 37kB dask.array<chunksize=(288, 32), meta=np.ndarray>
    alp/inner_lo_density_error     (time, npts_inner) float32 37kB dask.array<chunksize=(288, 32), meta=np.ndarray>
    alp/inner_lo_jsapeakpos        (time) float32 1kB dask.array<chunksize=(288,), meta=np.ndarray>
    ...                             ...
    alp/outer_up_timemid           (time, npts_outer) float32 74kB dask.array<chunksize=(288, 64), meta=np.ndarray>
    alp/outer_up_vfloat            (time, npts_outer) float32 74kB dask.array<chunksize=(288, 64), meta=np.ndarray>
    alp/outer_up_vfloat_error      (time, npts_outer) float32 74kB dask.array<chunksize=(288, 64), meta=np.ndarray>
    alp/outer_up_z                 (npts_outer) float32 256B dask.array<chunksize=(64,), meta=np.ndarray>
    alp/passnumber                 float32 4B dask.array<chunksize=(), meta=np.ndarray>
    alp/status                     float32 4B dask.array<chunksize=(), meta=np.ndarray>

### AMA

In [62]:
source = 'ama'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 26MB
Dimensions:              (time: 650001)
Coordinates:
  * time                 (time) float32 3MB -0.1 -0.1 -0.1 ... 1.2 1.2 1.2
Data variables:
    ama/n=2_amplitude    (time) float32 3MB dask.array<chunksize=(81251,), meta=np.ndarray>
    ama/n=2_frequency    (time) float32 3MB dask.array<chunksize=(81251,), meta=np.ndarray>
    ama/n=2_signal       (time) float32 3MB dask.array<chunksize=(81251,), meta=np.ndarray>
    ama/n=odd_amplitude  (time) float32 3MB dask.array<chunksize=(81251,), meta=np.ndarray>
    ama/n=odd_frequency  (time) float32 3MB dask.array<chunksize=(81251,), meta=np.ndarray>
    ama/n=odd_signal     (time) float32 3MB dask.array<chunksize=(81251,), meta=np.ndarray>
    ama/passnumber       (time) float32 3MB dask.array<chunksize=(650001,), meta=np.ndarray>
    ama/status           (time) float32 3MB dask.array<chunksize=(650001,), meta=np.ndarray>
    ama/version          (time) float32 3MB dask.array<chunksize=(650001,), meta=np.ndarray>

### AMB - Needs Review

AMB needs further tensorising to combine channels. Also requires including mapping r and z coordinates

In [78]:
source = 'amb'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

# df = make_channel_df(dataset)
# df.to_csv('amb.csv')

<xarray.Dataset> Size: 20MB
Dimensions:        (time: 67050, ccbv_channel: 36, ob_channel: 22,
                    fl_cc_channel: 5, fl_p_channel: 10)
Coordinates:
  * time           (time) float32 268kB -2.0 -2.0 -2.0 -2.0 ... 4.0 4.0 4.0 4.0
  * ccbv_channel   (ccbv_channel) <U10 1kB 'amb/ccbv01' ... 'amb/ccbv40'
  * ob_channel     (ob_channel) <U9 792B 'amb/obr01' 'amb/obr04' ... 'amb/obv18'
  * fl_cc_channel  (fl_cc_channel) <U11 220B 'amb/fl_cc03' ... 'amb/fl_cc09'
  * fl_p_channel   (fl_p_channel) <U12 480B 'amb/fl_p3l_4' ... 'amb/fl_p6l_1'
Data variables:
    amb/status     (time) float32 268kB dask.array<chunksize=(67050,), meta=np.ndarray>
    amb/version    (time) float32 268kB dask.array<chunksize=(67050,), meta=np.ndarray>
    ccbv           (ccbv_channel, time) float32 10MB dask.array<chunksize=(36, 67050), meta=np.ndarray>
    ob             (ob_channel, time) float32 6MB dask.array<chunksize=(22, 67050), meta=np.ndarray>
    fl_cc          (fl_cc_channel, time) float32 1MB dask.array<chunksize=(5, 67050), meta=np.ndarray>
    fl_p           (fl_p_channel, time) float32 3MB dask.array<chunksize=(10, 67050), meta=np.ndarray>

### AMC - Needs Review

In [82]:
source = 'amc'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

# df = make_channel_df(dataset)
# df.to_csv('amc.csv')

<xarray.Dataset> Size: 6MB
Dimensions:                (time: 30001)
Coordinates:
  * time                   (time) float32 120kB -2.0 -2.0 -2.0 ... 3.999 4.0 4.0
Data variables: (12/46)
    amc/efps_current       (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amc/error_field_02     (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amc/error_field_05     (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amc/p2il_coil_current  (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amc/p2il_feed_current  (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amc/p2iu_coil_current  (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>
    ...                     ...
    amc/p6u_current        (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amc/plasma_current     (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amc/sol_current        (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amc/status             (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amc/tf_current         (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amc/version            (time) float32 120kB dask.array<chunksize=(30001,), meta=np.ndarray>

### AMH - Needs Review

AMH also needs further tensorising, but we should check with Lucy about groups.

In [92]:
source = 'amh'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'

dt = load_source(path)
# dataset = group_signals_by_source(source, dt)
dataset = pipelines.get(source)(dt)
dataset

# df = make_channel_df(dataset)
# df.to_csv('amh.csv')

<xarray.Dataset> Size: 563kB
Dimensions:                  (time: 2201)
Coordinates:
  * time                     (time) float32 9kB 0.0 0.289 0.289 ... 0.329 0.329
Data variables: (12/63)
    amh/halo_ccu_2           (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>
    amh/halo_p2l_1           (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>
    amh/halo_p2l_2           (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>
    amh/halo_p2l_3           (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>
    amh/halo_p2l_4           (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>
    amh/halo_p2l_5           (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>
    ...                       ...
    amh/phalo_sum_p2l_inner  (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>
    amh/phalo_sum_p2l_outer  (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>
    amh/phalo_sum_p2u_inner  (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>
    amh/phalo_sum_p2u_outer  (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>
    amh/status               (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>
    amh/version              (time) float32 9kB dask.array<chunksize=(2201,), meta=np.ndarray>

### AMM - Needs Review

Another source that can be hevily tensorized

In [99]:
source = 'amm'
path = f'/common/tmp/sjackson/local_cache2/30420.zarr/{source}'
dt = load_source(path)
# dataset = group_signals_by_source(source, dt)
dataset = pipelines.get(source)(dt)
dataset
# dataset = tensorise_group(dataset, 'incon')
# dataset = tensorise_group(dataset, 'mid')
# dataset = tensorise_group(dataset, 'ring')
# dataset = tensorise_group(dataset, 'rodgr')
# dataset = tensorise_group(dataset, 'vertw')
# dataset = tensorise_group(dataset, 'lhorw')
# dataset = tensorise_group(dataset, 'uhorw')
# dataset

# df = make_channel_df(dataset)
# df.to_csv('amm.csv')

<xarray.Dataset> Size: 20MB
Dimensions:         (time: 30001, incon_channel: 10, mid_channel: 12,
                     ring_channel: 10, rodgr_channel: 12, vertw_channel: 8,
                     lhorw_channel: 6, uhorw_channel: 6)
Coordinates:
  * time            (time) float32 120kB -2.0 -2.0 -2.0 ... 4.0 4.001 4.001
  * incon_channel   (incon_channel) <U11 440B 'amm/incon1' ... 'amm/incon9'
  * mid_channel     (mid_channel) <U9 432B 'amm/mid1' 'amm/mid10' ... 'amm/mid9'
  * ring_channel    (ring_channel) <U10 400B 'amm/ring1' ... 'amm/ring9'
  * rodgr_channel   (rodgr_channel) <U11 528B 'amm/rodgr1' ... 'amm/rodgr9'
  * vertw_channel   (vertw_channel) <U10 320B 'amm/vertw1' ... 'amm/vertw8'
  * lhorw_channel   (lhorw_channel) <U10 240B 'amm/lhorw1' ... 'amm/lhorw6'
  * uhorw_channel   (uhorw_channel) <U10 240B 'amm/uhorw1' ... 'amm/uhorw6'
Data variables: (12/25)
    amm/botcol      (time) float64 240kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amm/endcrown_l  (time) float64 240kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amm/endcrown_u  (time) float64 240kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amm/p2larm1     (time) float64 240kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amm/p2larm2     (time) float64 240kB dask.array<chunksize=(30001,), meta=np.ndarray>
    amm/p2larm3     (time) float64 240kB dask.array<chunksize=(30001,), meta=np.ndarray>
    ...              ...
    mid             (mid_channel, time) float64 3MB dask.array<chunksize=(12, 30001), meta=np.ndarray>
    ring            (ring_channel, time) float64 2MB dask.array<chunksize=(10, 30001), meta=np.ndarray>
    rodgr           (rodgr_channel, time) float64 3MB dask.array<chunksize=(12, 30001), meta=np.ndarray>
    vertw           (vertw_channel, time) float64 2MB dask.array<chunksize=(8, 30001), meta=np.ndarray>
    lhorw           (lhorw_channel, time) float64 1MB dask.array<chunksize=(6, 30001), meta=np.ndarray>
    uhorw           (uhorw_channel, time) float64 1MB dask.array<chunksize=(6, 30001), meta=np.ndarray>

### AMS

In [103]:
source = 'ams'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 3GB
Dimensions:                     (dim_0: 2219, dim_1: 30, time: 2367)
Coordinates:
  * dim_0                       (dim_0) float32 9kB 0.0 1.0 ... 2.218e+03
  * dim_1                       (dim_1) float32 120B 0.0 1.0 2.0 ... 28.0 29.0
  * time                        (time) float32 9kB -0.047 -0.0465 ... 0.5975
Data variables: (12/57)
    ams/acoeff                  (time, dim_0, dim_1) float32 630MB dask.array<chunksize=(2367, 2219, 30), meta=np.ndarray>
    ams/beam_ok                 (time) float32 9kB dask.array<chunksize=(2367,), meta=np.ndarray>
    ams/ch                      (time, dim_0) float32 21MB dask.array<chunksize=(2367, 2219), meta=np.ndarray>
    ams/cosbeam                 (time, dim_0) float32 21MB dask.array<chunksize=(2367, 2219), meta=np.ndarray>
    ams/cpf_data                (time, dim_0) float32 21MB dask.array<chunksize=(2367, 2219), meta=np.ndarray>
    ams/cpf_error               (time, dim_0) float32 21MB dask.array<chunksize=(2367, 2219), meta=np.ndarray>
    ...                          ...
    ams/status                  (time) float64 19kB dask.array<chunksize=(2367,), meta=np.ndarray>
    ams/trans                   (time, dim_0) float32 21MB dask.array<chunksize=(2367, 2219), meta=np.ndarray>
    ams/version                 (time) float32 9kB dask.array<chunksize=(2367,), meta=np.ndarray>
    ams/viewstr                 (time, dim_0) float32 21MB dask.array<chunksize=(2367, 2219), meta=np.ndarray>
    ams/vx0                     (time, dim_0, dim_1) float32 630MB dask.array<chunksize=(2367, 2219, 30), meta=np.ndarray>
    ams/vy0                     (time, dim_0, dim_1) float32 630MB dask.array<chunksize=(2367, 2219, 30), meta=np.ndarray>

### ANB

In [104]:
source = 'anb'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 11MB
Dimensions:                         (time: 99921)
Coordinates:
  * time                            (time) float64 799kB -0.5 -0.5 ... 2.0 2.0
Data variables: (12/24)
    anb/passnumber                  (time) float64 799kB dask.array<chunksize=(99921,), meta=np.ndarray>
    anb/ss_full_power_data          (time) float32 400kB dask.array<chunksize=(99921,), meta=np.ndarray>
    anb/ss_full_power_error         (time) float32 400kB dask.array<chunksize=(99921,), meta=np.ndarray>
    anb/ss_half_power_data          (time) float32 400kB dask.array<chunksize=(99921,), meta=np.ndarray>
    anb/ss_half_power_error         (time) float32 400kB dask.array<chunksize=(99921,), meta=np.ndarray>
    anb/ss_source_gas_deuter_data   (time) float32 400kB dask.array<chunksize=(99921,), meta=np.ndarray>
    ...                              ...
    anb/sw_sum_power_data           (time) float32 400kB dask.array<chunksize=(99921,), meta=np.ndarray>
    anb/sw_sum_power_error          (time) float32 400kB dask.array<chunksize=(99921,), meta=np.ndarray>
    anb/sw_third_power_data         (time) float32 400kB dask.array<chunksize=(99921,), meta=np.ndarray>
    anb/sw_third_power_error        (time) float32 400kB dask.array<chunksize=(99921,), meta=np.ndarray>
    anb/tot_sum_power_data          (time) float32 400kB dask.array<chunksize=(99921,), meta=np.ndarray>
    anb/tot_sum_power_error         (time) float32 400kB dask.array<chunksize=(99921,), meta=np.ndarray>

### ANE

In [105]:
source = 'ane'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset


<xarray.Dataset> Size: 1MB
Dimensions:            (time: 32768)
Coordinates:
  * time               (time) float32 131kB -0.01 -0.00996 ... 1.301 1.301
Data variables:
    ane/co2            (time) float32 131kB dask.array<chunksize=(32768,), meta=np.ndarray>
    ane/density        (time) float32 131kB dask.array<chunksize=(32768,), meta=np.ndarray>
    ane/hene           (time) float32 131kB dask.array<chunksize=(32768,), meta=np.ndarray>
    ane/passnumber     (time) float32 131kB dask.array<chunksize=(32768,), meta=np.ndarray>
    ane/status         (time) float32 131kB dask.array<chunksize=(32768,), meta=np.ndarray>
    ane/status_detail  (time) float32 131kB dask.array<chunksize=(32768,), meta=np.ndarray>
    ane/version        (time) float32 131kB dask.array<chunksize=(32768,), meta=np.ndarray>

### ANT

In [106]:
source = 'ant'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset


<xarray.Dataset> Size: 20B
Dimensions:           (time: 1)
Coordinates:
  * time              (time) float32 4B 0.0
Data variables:
    ant/neutron_dose  (time) float32 4B dask.array<chunksize=(1,), meta=np.ndarray>
    ant/passnumber    (time) float32 4B dask.array<chunksize=(1,), meta=np.ndarray>
    ant/status        (time) float32 4B dask.array<chunksize=(1,), meta=np.ndarray>
    ant/version       (time) float32 4B dask.array<chunksize=(1,), meta=np.ndarray>

### ANU

In [107]:
source = 'anu'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 7MB
Dimensions:             (time: 194513)
Coordinates:
  * time                (time) float32 778kB -0.1025 -0.1 ... 1.388 1.393
Data variables:
    anu/errors          (time) float32 778kB dask.array<chunksize=(194513,), meta=np.ndarray>
    anu/neutrons        (time) float32 778kB dask.array<chunksize=(194513,), meta=np.ndarray>
    anu/neutrons_cb     (time) float32 778kB dask.array<chunksize=(95560,), meta=np.ndarray>
    anu/neutrons_count  (time) float32 778kB dask.array<chunksize=(194513,), meta=np.ndarray>
    anu/neutrons_dc     (time) float32 778kB dask.array<chunksize=(95563,), meta=np.ndarray>
    anu/passnumber      (time) float32 778kB dask.array<chunksize=(194513,), meta=np.ndarray>
    anu/status          (time) float32 778kB dask.array<chunksize=(194513,), meta=np.ndarray>
    anu/version         (time) float32 778kB dask.array<chunksize=(194513,), meta=np.ndarray>

### AOE - Needs Review

Requires further work! Time ranges are different!

In [112]:
source = 'aoe'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)

dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 21MB
Dimensions:         (time_full: 262144, time: 749850)
Coordinates:
  * time_full       (time_full) float32 1MB 2.384e-07 2.238e-06 ... 0.5243
  * time            (time) float32 3MB 0.27 0.27 0.27 0.27 ... 0.285 0.285 0.285
Data variables:
    aoe/co2_frac    (time_full) float32 1MB dask.array<chunksize=(65536,), meta=np.ndarray>
    aoe/fast_k      (time) float32 3MB dask.array<chunksize=(93732,), meta=np.ndarray>
    aoe/fast_ka     (time) float32 3MB dask.array<chunksize=(93732,), meta=np.ndarray>
    aoe/fast_sync   (time) float32 3MB dask.array<chunksize=(93732,), meta=np.ndarray>
    aoe/fast_u      (time) float32 3MB dask.array<chunksize=(93732,), meta=np.ndarray>
    aoe/k_band      (time_full) float32 1MB dask.array<chunksize=(65536,), meta=np.ndarray>
    aoe/ka_band     (time_full) float32 1MB dask.array<chunksize=(65536,), meta=np.ndarray>
    aoe/passnumber  float32 4B dask.array<chunksize=(), meta=np.ndarray>
    aoe/ramping     (time_full) float32 1MB dask.array<chunksize=(65536,), meta=np.ndarray>
    aoe/status      float32 4B dask.array<chunksize=(), meta=np.ndarray>
    aoe/u_band      (time_full) float32 1MB dask.array<chunksize=(65536,), meta=np.ndarray>
    aoe/version     float32 4B dask.array<chunksize=(), meta=np.ndarray>

### ARP

In [115]:
source = 'arp'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 4MB
Dimensions:          (time: 177305)
Coordinates:
  * time             (time) float32 709kB 0.0 0.15 0.15 ... 0.2914 0.2914 0.2914
Data variables:
    arp/passnumber   (time) float32 709kB dask.array<chunksize=(177305,), meta=np.ndarray>
    arp/rp_gap_efit  (time) float32 709kB dask.array<chunksize=(90745,), meta=np.ndarray>
    arp/rp_radius    (time) float32 709kB dask.array<chunksize=(91667,), meta=np.ndarray>
    arp/status       (time) float32 709kB dask.array<chunksize=(177305,), meta=np.ndarray>

### ASB

In [116]:
source = 'asb'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 2kB
Dimensions:           (time: 54)
Coordinates:
  * time              (time) float32 216B -0.015 0.0 0.002204 ... 0.8624 1.0
Data variables:
    asb/cii_dga       (time) float32 216B dask.array<chunksize=(54,), meta=np.ndarray>
    asb/oii_dga       (time) float32 216B dask.array<chunksize=(54,), meta=np.ndarray>
    asb/passnumber    (time) float64 432B dask.array<chunksize=(54,), meta=np.ndarray>
    asb/status        (time) float64 432B dask.array<chunksize=(54,), meta=np.ndarray>
    asb/svn_revision  (time) float32 216B dask.array<chunksize=(54,), meta=np.ndarray>

### ASM - Needs Review
Another one which needs to be tensorized

In [117]:
source = 'asm'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset
# df = make_channel_df(dataset)
# df.to_csv('asm.csv')

<xarray.Dataset> Size: 3MB
Dimensions:                  (time: 32164, sad_m_channel: 11)
Coordinates:
  * time                     (time) float32 129kB -2.0 -2.0 -2.0 ... 4.001 4.001
  * sad_m_channel            (sad_m_channel) <U12 528B 'asm/sad_m01' ... 'asm...
Data variables: (12/16)
    asm/hm_periods           (time) float32 129kB dask.array<chunksize=(32164,), meta=np.ndarray>
    asm/hm_rating            (time) float32 129kB dask.array<chunksize=(32164,), meta=np.ndarray>
    asm/out_nn_pattern       (time) float32 129kB dask.array<chunksize=(32164,), meta=np.ndarray>
    asm/out_nn_rating        (time) float32 129kB dask.array<chunksize=(32164,), meta=np.ndarray>
    asm/out_rating           (time) float32 129kB dask.array<chunksize=(32164,), meta=np.ndarray>
    asm/out_signal           (time) float32 129kB dask.array<chunksize=(32164,), meta=np.ndarray>
    ...                       ...
    asm/sad_d02_2-8          (time) float32 129kB dask.array<chunksize=(32164,), meta=np.ndarray>
    asm/sad_d03_4-10         (time) float32 129kB dask.array<chunksize=(32164,), meta=np.ndarray>
    asm/sad_d04_5-11         (time) float32 129kB dask.array<chunksize=(32164,), meta=np.ndarray>
    asm/sad_d05_6-12         (time) float32 129kB dask.array<chunksize=(32164,), meta=np.ndarray>
    asm/status               (time) float32 129kB dask.array<chunksize=(32164,), meta=np.ndarray>
    sad_m                    (sad_m_channel, time) float32 1MB dask.array<chunksize=(11, 32164), meta=np.ndarray>

### ASX - Needs Review

Time has two values, but these differ between variables.

In [122]:
source = 'asx'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)

dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 92B
Dimensions:                   (npts: 2)
Coordinates:
    time                      float32 4B 0.0
Dimensions without coordinates: npts
Data variables:
    asx/elm_freqs             (npts) float32 8B -0.01 -0.00998
    asx/elmy                  (npts) float32 8B -0.01 -0.00998
    asx/large_scale           (npts) float32 8B 0.0 0.0
    asx/low_cam_nos_sawt_inv  (npts) float32 8B 0.0 0.0
    asx/lower_false_inv_rad   (npts) float32 8B 0.0 0.0
    asx/modes                 (npts) float32 8B 0.0 0.0
    asx/passnumber            int32 4B dask.array<chunksize=(), meta=np.ndarray>
    asx/sawteeth              (npts) float32 8B 0.0 0.0
    asx/sawtooth_periods      (npts) float32 8B 0.0 0.0
    asx/status                int32 4B dask.array<chunksize=(), meta=np.ndarray>
    asx/up_cam_nos_sawt_inv   (npts) float32 8B 0.0 0.0
    asx/upper_false_inv_rad   (npts) float32 8B 0.0 0.0

### AYC

In [2]:
source = 'ayc'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

/home/rt2549/envs/fmast/lib/python3.9/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


NameError: name 'pipelines' is not defined

### AYE

In [162]:
source = 'aye'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = pipelines.get(source)(dt)
dataset

<xarray.Dataset> Size: 357kB
Dimensions:               (arb: 16, time: 138, radialindex: 16, spectralindex: 4)
Coordinates:
  * arb                   (arb) float32 64B 0.0 1.0 2.0 3.0 ... 13.0 14.0 15.0
  * time                  (time) float32 552B 0.0 0.00416 0.00832 ... 67.0 68.0
  * radialindex           (radialindex) float32 64B 1.0 2.0 3.0 ... 15.0 16.0
  * spectralindex         (spectralindex) float32 16B 1.0 2.0 3.0 4.0
Data variables: (12/34)
    angle                 (arb) float32 64B dask.array<chunksize=(16,), meta=np.ndarray>
    aspectra              (time, radialindex, spectralindex) float32 35kB dask.array<chunksize=(138, 16, 4), meta=np.ndarray>
    chi2                  (time, radialindex) float32 9kB dask.array<chunksize=(138, 16), meta=np.ndarray>
    covariance_ne_te      (time, radialindex) float32 9kB dask.array<chunksize=(138, 16), meta=np.ndarray>
    gauss_amplitude       (time, radialindex, spectralindex) float32 35kB dask.array<chunksize=(138, 16, 4), meta=np.ndarray>
    gauss_dclevel         (time, radialindex, spectralindex) float32 35kB dask.array<chunksize=(138, 16, 4), meta=np.ndarray>
    ...                    ...
    te_error              (time, radialindex) float32 9kB dask.array<chunksize=(138, 16), meta=np.ndarray>
    time_                 (time) float32 552B dask.array<chunksize=(138,), meta=np.ndarray>
    version_fibre         float32 4B dask.array<chunksize=(), meta=np.ndarray>
    version_poly          float32 4B dask.array<chunksize=(), meta=np.ndarray>
    version_raman         float32 4B dask.array<chunksize=(), meta=np.ndarray>
    version_spectrometer  float32 4B dask.array<chunksize=(), meta=np.ndarray>

### EFM - Needs Review
Needs more work

0 -1 is normalised psi

psi - psi_norm - areap_c
dim_0 - num_iterations - chisq_magnetic
psi - psi_norm -  elongpsi_c
dim_0 - fcoil_segs_n - fcoil_ang1
dim_0 - fcoil_segs_n - fcoil_ang2
dim_0 - fcoil_n - fcoil_c
dim_0 - fcoil_n - fcoil_chisq
dim_0 - ffprime_coefs_n - ffprime_coefs
dim_0 - magpr_n - fwtmp
dim_0 - silop_n - fwtsi
dim_0 - nr - gridr
dim_0 - nz - gridz
r - nr_prof - jr
dim_0 - limiter_n - limiterr
dim_0 - limiter_n - limiterz
dim_0 - magpr_n - magpr_ang
z/r - nz/nr - plasma_currrz

pprime_coefs - as ffprime
r - nr_prof - pr_c
r - nr_prof - psir
height/radius - nz/nr - psirz
psi - psi_norm - pwpsi_c
r - nr_prof - qr
r - nr_prof - rvals - rvals for the radial profiles

dim_0 - silop_n - silop_c
dim_0 - silop_n - silop_chisq

In [45]:
source = 'efm'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)

with Path('mappings/dim_names.json').open('r') as handle:
    dimension_mapping = json.load(handle)

dt.pop('fcoil_n')
dt.pop('fcoil_segs_n')
dt.pop('limitern')
dt.pop('magpr_n')
dt.pop('silop_n')
dt.pop('shot_number')


items = []
for key, ds in dt.items():
    name = ds.attrs['name']
    ds = ds.squeeze()

    old_dims = [(k, v) for k, v in ds.sizes.items()]

    if name in dimension_mapping:
        ds = ds.rename_dims(dimension_mapping[name])
        ds = ds.rename_vars(dimension_mapping[name])

    for dim_name in ds.sizes.keys():
        if (ds[dim_name] == 0).all():
            ds= ds.drop_vars(dim_name)
        

    new_dims = [(k, v) for k, v in ds.sizes.items()]
    item = dict(name=ds.attrs['uda_name'], old_dims=old_dims, new_dims=new_dims, description=ds.attrs['description'])
    items.append(item)

    ds.attrs['dims'] = list(ds.sizes.keys())
    dt[key] = ds

dataset = group_signals_by_source(source, dt)
dataset

df = pd.DataFrame(items)
df.to_csv('efm.csv')

### ESM - Needs Review

ibmp1_r_larmor and similar can be dropped if unity.

In [51]:
source = 'esm'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'

with Path('mappings/dim_names.json').open('r') as handle:
    dimension_mapping = json.load(handle)

dt = load_source(path)
items = []
for key, ds in dt.items():
    name = ds.attrs['name']
    ds = ds.squeeze()

    old_dims = [(k, v) for k, v in ds.sizes.items()]

    if name in dimension_mapping:
        ds = ds.rename_dims(dimension_mapping[name])
        ds = ds.rename_vars(dimension_mapping[name])

    for dim_name in ds.sizes.keys():
        if (ds[dim_name] == 0).all():
            ds= ds.drop_vars(dim_name)
        

    new_dims = [(k, v) for k, v in ds.sizes.items()]
    item = dict(name=ds.attrs['uda_name'], old_dims=old_dims, new_dims=new_dims, description=ds.attrs['description'])
    items.append(item)

    ds.attrs['dims'] = list(ds.sizes.keys())
    dt[key] = ds

dataset = group_signals_by_source(source, dt)

df = pd.DataFrame(items)
df.to_csv('esm.csv')
dataset

<xarray.Dataset> Size: 550kB
Dimensions:               (time: 52, fit_quality_n: 2, time_multi: 69,
                           time_ane: 32768, radius: 130, time_r_multi: 8970)
Coordinates:
  * time                  (time) float32 208B 0.03 0.035 0.04 ... 0.28 0.285
  * fit_quality_n         (fit_quality_n) float32 8B -999.0 -1e+03
  * time_multi            (time_multi) float32 276B 0.00416 0.00832 ... 0.2875
  * time_ane              (time_ane) float32 131kB -0.01 -0.00996 ... 1.301
  * radius                (radius) float32 520B 0.0 1.0 2.0 ... 128.0 129.0
  * time_r_multi          (time_r_multi) float32 36kB 0.2346 0.2362 ... 1.516
Data variables: (12/79)
    area_outin_ratio      (time) float32 208B dask.array<chunksize=(52,), meta=np.ndarray>
    av_t                  float32 4B dask.array<chunksize=(), meta=np.ndarray>
    dr_sep_in             (time) float32 208B dask.array<chunksize=(52,), meta=np.ndarray>
    dr_sep_out            (time) float32 208B dask.array<chunksize=(52,), meta=np.ndarray>
    efm_comb_ratingav     (fit_quality_n) float32 8B dask.array<chunksize=(2,), meta=np.ndarray>
    efm_comb_ratingmin    (fit_quality_n) float32 8B dask.array<chunksize=(2,), meta=np.ndarray>
    ...                    ...
    v_ion                 float32 4B dask.array<chunksize=(), meta=np.ndarray>
    v_ion_multi           (time_multi, radius) float32 36kB dask.array<chunksize=(69, 130), meta=np.ndarray>
    v_loop_dynamic        (time) float32 208B dask.array<chunksize=(52,), meta=np.ndarray>
    v_loop_static         (time) float32 208B dask.array<chunksize=(52,), meta=np.ndarray>
    w_dot                 (time) float32 208B dask.array<chunksize=(52,), meta=np.ndarray>
    x                     (time) float32 208B dask.array<chunksize=(52,), meta=np.ndarray>

### ESX

In [174]:
source = 'esx'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = group_signals_by_source(source, dt)
# dataset = tensorise_group(dataset, stem='sad_m')
dataset

<xarray.Dataset> Size: 48B
Dimensions:           (time: 2)
Coordinates:
  * time              (time) float32 8B 0.0 0.0
Data variables:
    lower_inv_psi     (time) float32 8B dask.array<chunksize=(2,), meta=np.ndarray>
    lower_inv_radius  (time) float32 8B dask.array<chunksize=(2,), meta=np.ndarray>
    passnumber        float32 4B dask.array<chunksize=(), meta=np.ndarray>
    status            float32 4B dask.array<chunksize=(), meta=np.ndarray>
    upper_inv_psi     (time) float32 8B dask.array<chunksize=(2,), meta=np.ndarray>
    upper_inv_radius  (time) float32 8B dask.array<chunksize=(2,), meta=np.ndarray>

### XSX

In [190]:
source = 'xsx'
path = f'/common/tmp/sjackson/local_cache2/30397.zarr/{source}'
dt = load_source(path)
dataset = group_signals_by_source(source, dt)
dataset = tensorise_group(dataset, 'hcam_l')
dataset = tensorise_group(dataset, 'hcam_u')
dataset = tensorise_group(dataset, 'tcam')
dataset

<xarray.Dataset> Size: 73MB
Dimensions:                          (time: 300000, dim_0: 96, dim_1: 2,
                                      hcam_l_channel: 18, hcam_u_channel: 18,
                                      tcam_channel: 18)
Coordinates:
  * time                             (time) float64 2MB -0.009999 ... 0.59
  * dim_0                            (dim_0) int32 384B 0 1 2 3 ... 92 93 94 95
  * dim_1                            (dim_1) int32 8B 0 1
  * hcam_l_channel                   (hcam_l_channel) <U9 648B 'hcam_l_1' ......
  * hcam_u_channel                   (hcam_u_channel) <U9 648B 'hcam_u_1' ......
  * tcam_channel                     (tcam_channel) <U7 504B 'tcam_1' ... 'tc...
Data variables:
    acq196_061_ch01                  (time) float32 1MB dask.array<chunksize=(75000,), meta=np.ndarray>
    acq196_147_96                    (time) float32 1MB dask.array<chunksize=(75000,), meta=np.ndarray>
    devices_d112_acq196_061_channel  (dim_0) float64 768B dask.array<chunksize=(96,), meta=np.ndarray>
    devices_d112_acq196_061_range    (dim_0, dim_1) float32 768B dask.array<chunksize=(96, 2), meta=np.ndarray>
    devices_d2_acq196_147_channel    (dim_0) int32 384B dask.array<chunksize=(96,), meta=np.ndarray>
    devices_d2_acq196_147_range      (dim_0, dim_1) float32 768B dask.array<chunksize=(96, 2), meta=np.ndarray>
    devices_limit                    (dim_0) float64 768B dask.array<chunksize=(96,), meta=np.ndarray>
    relay_monitor                    (time) float32 1MB dask.array<chunksize=(75000,), meta=np.ndarray>
    time1                            (time) float64 2MB dask.array<chunksize=(37500,), meta=np.ndarray>
    hcam_l                           (hcam_l_channel, time) float32 22MB dask.array<chunksize=(18, 300000), meta=np.ndarray>
    hcam_u                           (hcam_u_channel, time) float32 22MB dask.array<chunksize=(18, 300000), meta=np.ndarray>
    tcam                             (tcam_channel, time) float32 22MB dask.array<chunksize=(18, 300000), meta=np.ndarray>